## Data visualization

In [1]:
import os
import time

import fsspec
import geopandas as gpd
import holoviews as hv
import hvplot.pandas
import pandas as pd
import panel as pn
import xarray as xr
from dotenv import load_dotenv
from pyproj import CRS

load_dotenv(override=True)

# NOTE: access tokens to the data are available upon request from Floris Calkoen
sas_token = os.getenv("AZURE_STORAGE_SAS_TOKEN")
account_name = "coclico"
storage_options = {"account_name": account_name, "credential": sas_token}

In [2]:
TEST_PREDICTIONS_PREFIX = "az://typology/test/*.parquet"

fs = fsspec.filesystem("az", **storage_options)
files = fs.glob(TEST_PREDICTIONS_PREFIX)
test_layer_options = {f.split("/")[-1].replace(".parquet", ""): f for f in files}

In [3]:
file_browser = pn.widgets.Select(options=list(test_layer_options.keys()))
file_browser

Select(options=['2024-11-04T15:55:41.6781...], value='2024-11-04T15:55:41.67816...)

In [4]:
def fetch_data(fs, urlpath, storage_options):
    with fs.open(urlpath, mode="rb", **storage_options) as f:
        df = gpd.read_parquet(f)
    return df


df = fetch_data(fs, test_layer_options[file_browser.value], storage_options)

In [5]:
# Map symbols for shore_type
shore_type_markers = {
    "sandy_gravel_or_small_boulder_sediments": "diamond",
    "rocky_shore_platform_or_large_boulders": "square",
    "muddy_sediments": "circle",
    "no_sediment_or_shore_platform": "cross",
}

# Enhanced color map for coast_type
coast_type_colors = {
    "cliffed_or_steep": "#D72638",  # deep red
    "dune": "#F4D35E",  # sandy yellow
    "inlet": "#4F6D7A",  # soft blue
    "bedrock_plain": "#A9A9A9",  # solid gray
    "sediment_plain": "#8B5E3C",  # rich brown
    "wetland": "#3CAEA3",  # vibrant green
    "moderately_sloped": "#F08A5D",  # warm orange
}


# Add color and symbol mapping to the dataframe
df["coast_color"] = df["pred_coast_type"].map(coast_type_colors)
df["shore_marker"] = df["pred_shore_type"].map(shore_type_markers)
df2 = (
    df.assign(geometry=gpd.GeoSeries.from_xy(df.lon, df.lat, crs=4326))
    .copy()
    .dropna(subset="user")
    .reset_index(drop=True)[
        [
            "geometry",
            "pred_shore_type",
            "pred_coast_type",
            "coast_color",
            "shore_marker",
        ]
    ]
)

In [6]:
df2.hvplot.points(
    "geometry",
    color="coast_color",
    geo=True,
    size=50,
    hover_cols=["pred_shore_type", "pred_coast_type"],
    marker="shore_marker",
    tiles="ESRI",
)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (coast_color,shore_marker,pred_shore_type,pred_coast_type)

In [15]:
df

,uuid,user,transect_id,lon,lat,geometry,datetime_created,datetime_updated,shore_type,coastal_type,...,dist_b180,dist_b210,dist_b240,dist_b270,dist_b300,dist_b330,pred_shore_type,pred_coast_type,coast_color,shore_marker
0,414e72c268e6,floris-calkoen,cl02385s00tr01277839,-122.027344,36.953331,"LINESTRING (-122.03468 36.96016, -122.02001 36...",2024-10-25 13:47:37.412407+00:00,2024-10-25 14:51:30.490834+00:00,rocky_shore_platform_or_large_boulders,cliffed_or_steep,...,50556.0,62235.0,NaN,1604.0,3580.0,17454.0,rocky_shore_platform_or_large_boulders,cliffed_or_steep,#D72638,square
1,ce217ae136fa,floris-calkoen,cl02385s00tr01616939,-120.632126,35.062237,"LINESTRING (-120.62117 35.06215, -120.64309 35...",2024-10-25 15:15:30.469386+00:00,2024-10-25 15:15:30.469386+00:00,sandy_gravel_or_small_boulder_sediments,sediment_plain,...,NaN,NaN,76208.0,20838.0,NaN,NaN,sandy_gravel_or_small_boulder_sediments,dune,#F4D35E,diamond
2,1d88fb702268,floris-calkoen,cl02385s00tr01695439,-120.484016,34.483845,"LINESTRING (-120.47625 34.49015, -120.49179 34...",2024-10-25 15:16:16.420948+00:00,2024-10-25 15:16:16.420948+00:00,sandy_gravel_or_small_boulder_sediments,cliffed_or_steep,...,NaN,NaN,11342.0,5294.0,50249.0,NaN,sandy_gravel_or_small_boulder_sediments,cliffed_or_steep,#D72638,diamond
3,a13c868378a6,floris-calkoen,cl02502s07tr04039844,-124.139000,44.017696,"LINESTRING (-124.12654 44.01726, -124.15146 44...",2024-10-27 21:00:47.944265+00:00,2024-10-27 21:00:47.944265+00:00,no_sediment_or_shore_platform,inlet,...,NaN,NaN,NaN,6826.0,NaN,NaN,no_sediment_or_shore_platform,inlet,#4F6D7A,cross
4,cb910ec8c163,floris-calkoen,cl02502s07tr04078944,-124.206642,43.669701,"LINESTRING (-124.19426 43.66916, -124.21902 43...",2024-10-27 21:01:46.335033+00:00,2024-10-27 21:01:46.335033+00:00,no_sediment_or_shore_platform,inlet,...,NaN,NaN,NaN,112.0,NaN,NaN,rocky_shore_platform_or_large_boulders,cliffed_or_steep,#D72638,square
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,5ddda23c1d8f,floris-calkoen,cl43229s00tr00020430,25.314470,40.944202,"LINESTRING (25.31544 40.95318, 25.3135 40.93523)",2024-10-30 15:08:10.435711+00:00,2024-10-30 15:08:10.435711+00:00,sandy_gravel_or_small_boulder_sediments,inlet,...,NaN,NaN,243580.0,745.0,107426.0,52766.0,sandy_gravel_or_small_boulder_sediments,sediment_plain,#8B5E3C,diamond
144,47b188e7ef6e,floris-calkoen,cl43248s00tr00043369,24.691093,40.594143,"LINESTRING (24.68013 40.59749, 24.70206 40.5908)",2024-10-30 15:03:00.012150+00:00,2024-10-30 15:03:00.012150+00:00,no_sediment_or_shore_platform,cliffed_or_steep,...,16294.0,19413.0,22064.0,115.0,480.0,115833.0,no_sediment_or_shore_platform,cliffed_or_steep,#D72638,cross
145,6121097e80db,floris-calkoen,cl43608s01tr00034462,23.769505,35.664459,"LINESTRING (23.75951 35.66829, 23.77949 35.66063)",2024-10-30 15:29:36.134428+00:00,2024-10-30 15:29:36.134428+00:00,sandy_gravel_or_small_boulder_sediments,cliffed_or_steep,...,4212.0,3014.0,895.0,428.0,NaN,223342.0,rocky_shore_platform_or_large_boulders,cliffed_or_steep,#D72638,square
146,f597b8a89eeb,floris-calkoen,cl43738s00tr00006847,25.100788,41.005516,"LINESTRING (25.102 41.01448, 25.09957 40.99656)",2024-10-30 15:06:19.791108+00:00,2024-10-30 15:06:19.791108+00:00,sandy_gravel_or_small_boulder_sediments,sediment_plain,...,NaN,NaN,NaN,99.0,4167.0,206494.0,sandy_gravel_or_small_boulder_sediments,dune,#F4D35E,diamond
